# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [1]:
# Setup
import os
from dotenv import load_dotenv

# Inline env (optional). Prefer .env in project root.
os.environ.setdefault("LANGSMITH_TRACING", "true")
os.environ.setdefault("LANGSMITH_PROJECT", "Langsmith_intro")  # personalize project name
os.environ.setdefault("GEMINI_API_KEY", os.getenv("GEMINI_API_KEY", ""))
os.environ.setdefault("LANGSMITH_API_KEY", os.getenv("LANGSMITH_API_KEY", ""))

# Load from .env if present
load_dotenv(dotenv_path="../../.env", override=True)
print("Tracing:", os.getenv("LANGSMITH_TRACING"), "Project:", os.getenv("LANGSMITH_PROJECT"))

# Install Gemini client if needed
try:
    import google.generativeai as genai  # noqa: F401
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "google-generativeai", "-q"])


Tracing: true Project: nischala-personal-project


In [2]:
# Or you can use a .env file
#from dotenv import load_dotenv
#load_dotenv(dotenv_path="../../.env", override=True)

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [5]:
import sys
import os
import nest_asyncio
from langsmith import traceable

# Add the path where utils.py is located
sys.path.append("module_0")

from utils import get_vector_db_retriever  # now it should work

import google.generativeai as genai

# Configure Gemini
genai.configure(api_key=os.getenv("GEMINI_API_KEY", ""))


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from typing import List
import nest_asyncio
from langsmith import traceable
from utils import get_vector_db_retriever
import google.generativeai as genai
import os

# Gemini config
genai.configure(api_key=os.getenv("GEMINI_API_KEY", ""))

MODEL_PROVIDER = "google"
MODEL_NAME = "models/gemini-2.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="retriever", metadata={"owner": "Nischala", "vectordb": "sklearn"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="llm", metadata={"ls_provider": MODEL_PROVIDER, "ls_model_name": MODEL_NAME})
def call_gemini(messages: List[dict]):
    # Convert messages to single prompt
    prompt = "\n\n".join([m["content"] for m in messages])
    model = genai.GenerativeModel(MODEL_NAME)
    resp = model.generate_content(prompt)

    # Minimal adapter to keep .choices[0].message.content API
    class Obj:
        class ChoicesMsg:
            def __init__(self, content):
                self.message = type("Msg", (), {"content": content})
        def __init__(self, text):
            self.choices = [self.ChoicesMsg(text)]
    return Obj(resp.text)

@traceable(run_type="chain", metadata={"app_version": APP_VERSION})
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"},
    ]
    return call_gemini(messages)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content



@traceable handles the RunTree lifecycle for you!

In [7]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the `@traceable` decorator, simply decorate any Python function with `@traceable`. You must also set the `LANGSMITH_TRACING` environment variable to 'true' and the `LANGSMITH_API_KEY` environment variable to your API key. This setup allows traces to be logged to LangSmith.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [8]:
from langsmith import traceable
from typing import List
import google.generativeai as genai
import os

# Gemini setup (make sure GEMINI_API_KEY is in your env)
genai.configure(api_key=os.getenv("GEMINI_API_KEY", ""))

@traceable(run_type="retriever", metadata={"vectordb": "sklearn"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain", metadata={"app_version": APP_VERSION})
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"},
    ]
    return call_gemini(messages)

@traceable(run_type="llm", metadata={"ls_model_name": MODEL_NAME, "ls_provider": MODEL_PROVIDER})
def call_gemini(messages: List[dict], model: str = MODEL_NAME):
    # Convert OpenAI-like messages into single Gemini prompt
    prompt = "\n\n".join([m["content"] for m in messages])
    model = genai.GenerativeModel(MODEL_NAME)
    resp = model.generate_content(prompt)

    # Adapter to mimic OpenAI's .choices[0].message.content
    class Obj:
        class ChoicesMsg:
            def __init__(self, content):
                self.message = type("Msg", (), {"content": content})
        def __init__(self, text):
            self.choices = [self.ChoicesMsg(text)]
    return Obj(resp.text)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [9]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

The provided context shows how to customize the run name of a `traceable` function using `{ name: "parentTraceable" }`. You can add metadata to a `generateText` call by including it in the `experimental_telemetry.metadata` option. The context does not explicitly demonstrate how to pass generic metadata directly to the `traceable` function's configuration itself.


You can also add metadata at runtime!

In [10]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

Metadata can be added to traces dynamically at runtime. For AI SDK, you can include metadata within the `experimental_telemetry` field of a `generateText` call. For LangChain runnables, this can be achieved using invocation parameters, with the metadata then inherited by all child runnables.


##### Let's take a look in LangSmith!